[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab6/lab6-parte2.ipynb)
# Práctica 6: Redes neuronales convolucionales - Complicando la CNN


### Pre-requisitos. Instalar paquetes
Usaremos las mismas librerías que para la parte 1. En esta ocasión utilizaremos la GPU, así que declaramos la variable `device`. Recuerda enviar tus modelos al device cuando los instancies (`m = MiModelo().to(device)`) y de enviar también los tensores de datos en el bucle de entrenamiento (`X_batch = X_batch.to(device)` y lo mismo para las etiquetas).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import transforms
import numpy as np
import random
import os
from matplotlib import pyplot as plt

# Semilla para reproducibilidad
seed = 1234567
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Carga del conjunto de datos

En esta ocasión trabajaremos con el conjunto de imágenes *cifar10*, que son imágenes con tres canales.

In [ ]:
# Los datos de CIFAR se estandarizan utilizando su media y desviación típicas, que aquí vienen precalculadas.
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    # TODO - Usa transforms.Normalize para estandarizar
])

train_val = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

# TODO - Divide train_val en train y val (80/20)

# TODO - Crea los DataLoaders
batch_size = 128
train_loader = ...
val_loader = ...
test_loader = ...

# Mostramos un ejemplo del conjunto
images, labels = next(iter(train_loader))
print(images.shape)  # [128, 3, 32, 32]
print(labels.shape)  # [128]

# Para mostrar los datos con pyplot hay que deshacer la normalización y poner los canales como última dimensión del tensor
def unnormalize(img):
    img = img.permute(1,2,0)  # CHW -> HWC
    img = img * torch.tensor(cifar10_std) + torch.tensor(cifar10_mean)
    return img.clamp(0,1)

plt.imshow(unnormalize(images[0]))
plt.xlabel(labels[0].item())
plt.show()


## Creando el modelo y entrenando el modelo
Para este problema vamos a utilizar la siguiente arquitectura:
1. [Convolución 2D](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) de 32 filtros y tamaño de kernel 3, con activación ReLU
1. [Pooling 2D](https://docs.pytorch.org/docs/2.8/generated/torch.nn.MaxPool2d.html) tomando el máximo de cada grupo de 2x2
1. [Convolución 2D](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) de 64 filtros y tamaño de kernel 3, con activación ReLU
1. [Pooling 2D](https://docs.pytorch.org/docs/2.8/generated/torch.nn.MaxPool2d.html) tomando el máximo de cada grupo de 2x2
1. [Convolución 2D](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) de 64 filtros y tamaño de kernel 3, con activación ReLU
1. Capa Densa (requiere aplanado previo) de 64 unidades y activación ReLU
1. Capa de salida

Define el modelo y haz el entrenamiento como en la anterior parte.

In [ ]:
# TODO Define el modelo
# TODO Define las funciones de aprendizaje y evaluación
# TODO Realiza el entrenamiento
# TODO Examina las curvas de entrenamiento y mide el rendimiento en test

Si todo ha ido bien, deberías haber obtenido una preción en test de al menos un 60%, lo cual no es desdeñable para una red sencilla.

## Mejora del rendimiento
Vuelve sobre la arquitectura del modelo e incluye capas de [BatchNorm2d](https://docs.pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) después de cada capa convolucional. Esto hará que las salidas de esa capa se estandaricen para que sigan una distribución N(0,1) (la operación de estandarización se incluye en el grafo y, por tanto, el cómputo de los gradientes). El efecto de esta capa es que se evitará que el gradiente del lote tenga una gran componente solo dedicada a acercar las salidas a la media/desviación de las muestras en cada capa. En consecuencia, el aprendizaje se acelera.

### Ejercicios
 - Repite el entrenamiento con la nueva arquitectura. ¿Qué efecto has notado? ¿Puedes mejorar el rendimiento en test utilizando conceptos vistos en Laboratorios anteriores?
 - Prueba distintas arquitecturas e intenta mejorar el rendimiento en test.

In [ ]:
# TODO Define el nuevo modelo
# TODO Define las funciones de aprendizaje y evaluación
# TODO Realiza el entrenamiento
# TODO Examina las curvas de entrenamiento y mide el rendimiento en test